# Creating Postgres DB Environment and Loading Data

In this notebook, we will create our postgres database infrastructure and load our cleaned data into the database

## Imports + Load Environment Variables

In [37]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

True

# Create the Database Engine

In [38]:
# Load DB URL from .env file
DATABASE_URL = os.getenv("DATABASE_URL")

if DATABASE_URL is None:
    raise ValueError("DATABASE_URL not found. Check your .env file.")

# Establish connection
engine = create_engine(DATABASE_URL)

# Test
with engine.connect() as conn:
    print("Connected to Neon successfully!")

Connected to Neon successfully!


## Load Cleaned CSVs

In [39]:
# Load Console Sales Data into Pandas dataframe
df_consoles = pd.read_csv('../data/processed/console_data_clean.csv')

# Load Game Sales Data into Pandas dataframe
df_games = pd.read_csv('../data/processed/game_data_clean.csv')

# Check
df_consoles.head(), df_games.head()

(    console_name  generation manufacturer  released_year  discontinued_year  \
 0       Xbox 360           7    Microsoft           2005               2016   
 1  PlayStation 3           7         Sony           2006               2017   
 2            Wii           7     Nintendo           2006               2013   
 3          Wii U           8     Nintendo           2012               2017   
 4  PlayStation 4           8         Sony           2013               2021   
 
    units_sold(m)  
 0           84.0  
 1           87.0  
 2          101.0  
 3           14.0  
 4          117.0  ,
                                       title   console_name  \
 0                            Gran Turismo 5  PlayStation 3   
 1            Uncharted 3: Drake's Deception  PlayStation 3   
 2                            The Last of Us  PlayStation 3   
 3                            God of War III  PlayStation 3   
 4  Metal Gear Solid 4: Guns of the Patriots  PlayStation 3   
 
                 

*Important Note*: We need to convert the release_date back to type 'datetime'

In [40]:
# CSV dates are loaded as type 'object' in Pandas
df_games['release_date'] = pd.to_datetime(df_games['release_date']).dt.date

# Check
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5685 entries, 0 to 5684
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          5685 non-null   object 
 1   console_name   5685 non-null   object 
 2   publisher      5685 non-null   object 
 3   developer      5685 non-null   object 
 4   release_date   5685 non-null   object 
 5   units_sold(m)  5685 non-null   float64
 6   platform       5685 non-null   object 
 7   is_exclusive   5685 non-null   bool   
dtypes: bool(1), float64(1), object(6)
memory usage: 316.6+ KB


## Push Data to Postgres Database (Neon)

In [41]:
# Game Sales
with engine.begin() as connection:
    df_games.to_sql(
        name="game_sales",
        con=engine,
        if_exists="replace",
        index=False
    )

In [42]:
# Console Sales
with engine.begin() as connection:
    df_consoles.to_sql(
        name="console_sales",
        con=engine,
        if_exists="replace",
        index=False
    )

## Test and Verify

Check and compare number of rows to ensure the data loaded successfully!

In [43]:
# Game Sales Check
row_count = pd.read_sql(
    "SELECT COUNT(*) AS row_count FROM game_sales;",
    engine
)["row_count"].iloc[0]

if df_games.shape[0] == row_count:
    print("Game sales table loaded correctly!")
else:
    print("Mismatch between CSV and SQL!")

Game sales table loaded correctly!


In [44]:
# Console Sales Check
row_count = pd.read_sql(
    "SELECT COUNT(*) AS row_count FROM console_sales;",
    engine
)["row_count"].iloc[0]

if df_consoles.shape[0] == row_count:
    print("Console sales table loaded correctly!")
else:
    print("Mismatch between CSV and SQL!")

Console sales table loaded correctly!
